In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
import os
import sys
sys.path.append('..')

In [ ]:
import pandas as pd

In [ ]:
from pyMultiOmics.common import set_log_level_warning
_ = set_log_level_warning()

In [ ]:
from pyMultiOmics.loader import process_affinity_data
from pyMultiOmics.base import SingleOmicsData, MultiOmicsData
from pyMultiOmics.constants import CIC_COMPOUNDS, INFERENCE_T_TEST
from pyMultiOmics.analysis import AnalysisPipeline

# Data Pre-processing

The CiC Affinity Biomarkers data was read from the specified Excel file. To preprocess the data, analytes that fall below the Limit of Detection and appear in less than 50% of the samples in each plate were removed. Missing values for each analyte were replaced with the minimum value present. Batch correction was performed using [pyComBat](https://github.com/epigenelabs/pyComBat) to eliminate batch effects across plates. Subsequently, a logarithmic transformation was applied, followed by standardization to ensure each feature has a mean of 0 and a variance of 1. The transformed and standardized data is displayed in the heatmap below.

In [ ]:
# This cell is tagged 'parameters'
file_name = None

In [ ]:
print(file_name)

In [ ]:
filter_method = 'batch'
filter_thresh = 0.50
correct_batch = True

In [ ]:
data_df, sample_metadata_df, feature_metadata_df = process_affinity_data(
    file_name, filter_method=filter_method, filter_thresh=filter_thresh, correct_batch=correct_batch
)
print('Number of analytes =', data_df.shape[0])
print('Number of samples =', data_df.shape[1])
print('Groups =', sample_metadata_df['group'].unique().tolist())

In [ ]:
cic_data = SingleOmicsData(CIC_COMPOUNDS, data_df, sample_metadata_df, feature_annot_df=feature_metadata_df)
mo = MultiOmicsData()
mo.add_data([cic_data])

In [ ]:
normalise = 'standard'
log = True

In [ ]:
# This cell is tagged 'parameters'
PCA_components = 5

In [ ]:
dtype = CIC_COMPOUNDS
return_fig = True

ap = AnalysisPipeline(mo, None)
_ = ap.heatmap(dtype, normalise=normalise, log=log, return_fig=return_fig)

The transformed and scaled data was subjected to Principal Component Analysis using 5 components. A scatterplot displaying the projection of samples onto different components is presented below.

In [ ]:
_, design_df = ap.multi_omics_data.get_dfs(dtype)
pcs = ap.scatter_PCA(dtype, normalise=normalise, log=log, n_components=PCA_components, hue=design_df['Plate number'], return_fig=return_fig)

# Clustering Using k-means

K-means clustering is utilised to identify similar clusters, either based on samples or analytes. The best K (number of clusters) is automatically chosen by identifying turning point (the elbow) in the silhouette score plot. This allows for the identification of groups of analytes with similar intensity values, or samples that share similar characteristics.

## Clustering of Samples

Below are the results of the k-means clustering performed on the samples.





In [ ]:
kind = 'samples'
k = None

In [ ]:
pcs = ap.PCA(dtype, normalise=normalise, log=log, n_components=5, kind=kind)

In [ ]:
cluster_labels, cluster_order, centroids, silhouette_scores = ap.cluster(dtype, normalise=normalise, log=log, return_fig=return_fig, kind=kind, k=k, pcs=pcs)

### Principal Component Analysis (PCA)

A plot of the PCA projection of samples is displayed below, with cluster labels represented by different colours based on the best number of clusters selected using k-means clustering. The samples in different groups are separated by different shapes.

In [ ]:
_, design_df = ap.multi_omics_data.get_dfs(dtype)
pcs = ap.PCA(dtype, normalise=normalise, log=log, n_components=PCA_components, style=design_df['group'], hue=cluster_labels, return_fig=return_fig, kind=kind)

### Heatmap

The heatmap displayed below is organised by cluster labels, showcasing the clustering patterns of the samples.

In [ ]:
_ = ap.heatmap(dtype, normalise=normalise, log=log, return_fig=return_fig, kind=kind, cluster_order=cluster_order)

## Clustering of Analytes

Below are the results of the k-means clustering performed on the analytes.

In [ ]:
kind = 'features'
k = None

In [ ]:
pcs = ap.PCA(dtype, normalise=normalise, log=log, n_components=5, kind=kind)

In [ ]:
cluster_labels, cluster_order, centroids, silhouette_scores = ap.cluster(dtype, normalise=normalise, log=log, return_fig=return_fig, kind=kind, k=k, pcs=pcs)

### Principal Component Analysis (PCA)

A plot of the PCA projection of analytes is displayed below, with cluster labels represented by different colours.

In [ ]:
_, design_df = ap.multi_omics_data.get_dfs(dtype)
pcs = ap.PCA(dtype, normalise=normalise, log=log, n_components=PCA_components, hue=cluster_labels, return_fig=return_fig, kind=kind)

### Heatmap

The heatmap displayed below is organized by cluster labels, showcasing the clustering patterns of the analytes.

In [ ]:
_ = ap.heatmap(dtype, normalise=normalise, log=log, return_fig=return_fig, kind=kind, cluster_order=cluster_order)

# Case-vs-control Analysis

In this analysis, we perform a case-control comparison. Specifically, we conduct t-tests to compare the means of the case and control groups, as defined below. To control for multiple testing, we apply the Benjamini-Hochberg method to correct the False Discovery Rate (FDR).

In [ ]:
print('Groups =', sample_metadata_df['group'].unique().tolist())

In [ ]:
# This cell is tagged 'parameters'
case_group = 'disease'
control_group = 'control'

In [ ]:
de_method = INFERENCE_T_TEST
ap.run_de(de_method, dtype, case_group, control_group)
de_df = ap.get_de_results(dtype, case_group, control_group, de_method)

## Volcano Plot

The following Volcano Plot presents the results of T-tests, with a default p-value threshold of 0.05. Analytes were filtered using the Interquartile Range (IQR) method to remove outliers. This involved calculating the IQR as the difference between the 75th and 25th percentiles of the fold change (FC) values. Upper and lower bounds were then defined as 1.5 times the IQR above the 75th percentile and below the 25th percentile, respectively. Any analytes with FC values outside of these bounds were identified as outliers and removed from the analysis.

The top-10 analytes with the largest and smallest fold changes, which were also significantly different between the case and control groups, are highlighted in the volcano plot. Additionally, the top-20 significant analytes by fold change are reported in the tables that follow the plot. This information potentially highlights interesting biomarkers associated with the case and control groups.

In [ ]:
p_value_colname = 'padj_%s_vs_%s' % (case_group, control_group)
fc_colname = 'FC_%s_vs_%s' % (case_group, control_group)

In [ ]:
# This cell is tagged 'parameters'
p_value_thresh = 0.05
fc_iqr_thresh = 1.5
volcano_top_n = 10
report_top_n = 20

In [ ]:
ap.volcano(de_df, p_value_colname, p_value_thresh, fc_colname, fc_iqr_thresh=fc_iqr_thresh, top_n=volcano_top_n)

## Significantly-changing Analytes Ordered by Fold Changes (Descending)

Below is a list of analytes that have shown significant changes in abundance. The list is sorted in descending order based on their fold changes.

In [ ]:
fc_sort_order = 'desc'
sorted_df_asc = ap.de_sort_and_filter(de_df, p_value_colname, p_value_thresh, fc_colname, 
                                      fc_sort_order=fc_sort_order, top_n=report_top_n, fc_iqr_thresh=fc_iqr_thresh)

In [ ]:
merged_df = pd.merge(sorted_df_asc, cic_data.feature_annot_df[['Analyte Class', 'Analyte Name']], left_index=True, right_index=True, how='left')
merged_df

## Significantly-changing Analytes Ordered by Fold Changes (Ascending)

Below is a list of analytes that have shown significant changes in abundance. The list is sorted in ascending order based on their fold changes.

In [ ]:
fc_sort_order = 'asc'
sorted_df_desc = ap.de_sort_and_filter(de_df, p_value_colname, p_value_thresh, fc_colname, 
                                      fc_sort_order=fc_sort_order, top_n=report_top_n, fc_iqr_thresh=fc_iqr_thresh)

In [ ]:
merged_df = pd.merge(sorted_df_desc, cic_data.feature_annot_df[['Analyte Class', 'Analyte Name']], left_index=True, right_index=True, how='left')
merged_df